## S3 데이터 삭제

In [24]:
!aws s3 ls gonsoo-share/

                           PRE cifar10-output/


In [23]:
!aws s3 rm s3://gonsoo-share/ --recursive

delete: s3://gonsoo-share/cifar10-output/byoc-cifar10-2021-02-27-03-23-30-059/profiler-output/system/incremental/2021022703/1614396300.algo-1.json
delete: s3://gonsoo-share/cifar10-output/byoc-cifar10-2021-02-27-03-23-30-059/rule-output/ProfilerReport-1614396210/profiler-output/profiler-report.html
delete: s3://gonsoo-share/cifar10-output/byoc-cifar10-2021-02-27-03-23-30-059/rule-output/ProfilerReport-1614396210/profiler-output/profiler-reports/Dataloader.json
delete: s3://gonsoo-share/cifar10-output/byoc-cifar10-2021-02-27-03-23-30-059/rule-output/ProfilerReport-1614396210/profiler-output/profiler-report.ipynb
delete: s3://gonsoo-share/cifar10-output/byoc-cifar10-2021-02-27-03-23-30-059/rule-output/ProfilerReport-1614396210/profiler-output/profiler-reports/GPUMemoryIncrease.json
delete: s3://gonsoo-share/cifar10-output/byoc-cifar10-2021-02-27-03-23-30-059/rule-output/ProfilerReport-1614396210/profiler-output/profiler-reports/CPUBottleneck.json
delete: s3://gonsoo-share/cifar10-output/

## model_dir 확인

In [1]:
import tensorflow as tf

In [4]:
%%time
!mkdir -p logs
!python training_script/cifar10_keras_tf2.py --model_dir ./logs \
                                         --train data/train \
                                         --validation data/validation \
                                         --eval data/eval \
                                         --epochs 1
!rm -rf logs

2021-02-27 02:22:31.027957: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2021-02-27 02:22:31.052832: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3000000000 Hz
2021-02-27 02:22:31.054517: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55de23cb7690 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-02-27 02:22:31.054536: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-02-27 02:22:31.054670: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
args.model_dir:  ./logs
Train for 312 steps, validate for 78 steps
312/312 [==============================] - 29s 92ms/step - loss: 1.8570 - accuracy: 0.3176 - val_loss: 1.6851 - val_accu

In [5]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

In [6]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1, 
                       train_instance_type='local')
estimator.fit({'train': 'file://data/train',
               'validation': 'file://data/validation',
               'eval': 'file://data/eval'})

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Building with native build. Learn about native build in Compose here: https://docs.docker.com/go/compose-native-build/
Creating bbkpzwozlg-algo-1-sjd53 ... 
Creating bbkpzwozlg-algo-1-sjd53 ... done
Attaching to bbkpzwozlg-algo-1-sjd53
bbkpzwozlg-algo-1-sjd53 | 2021-02-27 02:28:36,852 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
bbkpzwozlg-algo-1-sjd53 | 2021-02-27 02:28:36,859 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
bbkpzwozlg-algo-1-sjd53 | 2021-02-27 02:28:37,050 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
bbkpzwozlg-algo-1-sjd53 | 2021-02-27 02:28:37,063 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
bbkpzwozlg-algo-1-sjd53 | 2021-02-27 02:28:37,076 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
bbkpzwozlg-algo-1-sjd53 | 2021-02-27 02:28:37,084 sagemaker-containers INFO     Invoking user script
bbkpzwozlg-algo-

## output bucket 테스트

In [7]:
output_bucket = 'gonsoo-share'
output_prefix = 'cifar10-output'
s3_output_path = f's3://{output_bucket}/{output_prefix}'
print(s3_output_path)

s3://gonsoo-share/cifar10-output


In [8]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='training_script',
                       output_path = s3_output_path,                       
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       script_mode=True,
                       hyperparameters={'epochs' : 2},
                       train_instance_count=1, 
                       train_instance_type='local')


estimator.fit({'train': 'file://data/train',
               'validation': 'file://data/validation',
               'eval': 'file://data/eval'})

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Building with native build. Learn about native build in Compose here: https://docs.docker.com/go/compose-native-build/
Creating 964xo1y87g-algo-1-bi92h ... 
Creating 964xo1y87g-algo-1-bi92h ... done
Attaching to 964xo1y87g-algo-1-bi92h
964xo1y87g-algo-1-bi92h | 2021-02-27 02:32:08,835 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
964xo1y87g-algo-1-bi92h | 2021-02-27 02:32:08,841 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
964xo1y87g-algo-1-bi92h | 2021-02-27 02:32:08,990 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
964xo1y87g-algo-1-bi92h | 2021-02-27 02:32:09,003 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
964xo1y87g-algo-1-bi92h | 2021-02-27 02:32:09,016 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
964xo1y87g-algo-1-bi92h | 2021-02-27 02:32:09,024 sagemaker-containers INFO     Invoking user script
964xo1y87g-algo-